In [2]:
import requests
import uuid
from RandomFunctions import Random_FHIR_Person
from RandomFunctions import Random_FHIR_ServiceRequest
from requests.auth import HTTPBasicAuth
from fhir.resources.R4B.bundle import Bundle

#fhir_server_url = 'https://spark.incendi.no/fhir'
#fhir_server_url = 'https://server.fire.ly'
#fhir_server_url = 'http://fhir.oridashi.com.au'
#fhir_server_url = 'https://demo.kodjin.com/fhir'
#Works R5# fhir_server_url = 'http://fhir.training.hl7.org.au/fhir'
    #fhir_server_url = 'https://demo.pathling.app/fhir'
    #fhir_server_url = 'https://fhir.hl7.org.au/ereq/fhir/DEFAULT' #sparked
fhir_server_url = "https://pyroserver.azurewebsites.net/pyro" # sonic


headers = {'Content-Type': 'application/fhir+json'}

# Create a patient
PersonGenerator = Random_FHIR_Person()
ServiceGenerator = Random_FHIR_ServiceRequest()


ValueSet Expansion: 3548 members
ValueSet Expansion: 1278 members


In [21]:
# Function to create a requisition
def create_requisition(value):
    return {
        "type": {
            "coding": [
                {
                    "system": "http://terminology.hl7.org/CodeSystem/v2-0203",
                    "code": "PGN",
                    "display": "Placer Group Number"
                }
            ]
        },
        "system": "https://github.com/MattCordell/callistemon",
        "value": value,  # Use the passed-in value
        "assigner": {
            "display": "Callistemon Placer"
        }
    }

In [42]:
import uuid
import requests
import random  # Import random for generating a random number
from fhir.resources.R4B.bundle import Bundle
from fhir.resources.R4B.narrative import Narrative
from fhir.resources.R4B.patient import Patient
from fhir.resources.R4B.practitioner import Practitioner
from fhir.resources.R4B.servicerequest import ServiceRequest
import time
import datetime

# Function to create a narrative
def create_narrative(resource):
    return Narrative(
        status="generated",
        div=f"<div xmlns='http://www.w3.org/1999/xhtml'>{resource.__class__.__name__} resource</div>"
    )

# Loop through 1as many times as you want to generate and post randomised data
for _ in range(5):
    
    # Details of the requisition group/episode
    current_timestamp = datetime.datetime.now().strftime('%d%H%M%S')  # Generate timestamp
    newRequesition = create_requisition(f"CAL-{current_timestamp}")
    num_requests = random.randint(1, 3)  # Number of serviceRequests within a single episode
    
    
    # Generate some randomised data
    patient = PersonGenerator.NewPatient()
    practitioner = PersonGenerator.NewPractitioner()
    # Ensure the IDs are valid UUIDs
    patient.id = str(uuid.uuid4())
    practitioner.id = str(uuid.uuid4())
    # Add narrative to each resource
    patient.text = create_narrative(patient)
    practitioner.text = create_narrative(practitioner)
    


    service_requests = []

    for i in range(num_requests):
        # Generate a new ServiceRequest
        referral = ServiceGenerator.NewRandomRequest(patient=patient, requester=practitioner)
        referral.id = str(uuid.uuid4())
        referral.text = create_narrative(referral)

        # Ensure the references in the referral are correctly set
        referral.subject.reference = f'urn:uuid:{patient.id}'
        referral.requester.reference = f'urn:uuid:{practitioner.id}'

        # Add a requisition with a unique value
        current_timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
        referral.requisition = newRequesition

        # Add the referral to the list
        service_requests.append({
            'fullUrl': f'urn:uuid:{referral.id}',
            'resource': referral,
            'request': {'method': 'PUT', 'url': f'ServiceRequest/{referral.id}'}
        })
    
    # Post a bundle with ServiceRequest and referenced patient+requester
    bundle = Bundle(
        type='transaction',
        id=str(uuid.uuid4()),
        
        
        entry=[
            {
                'fullUrl': f'urn:uuid:{patient.id}',
                'resource': patient,
                'request': {'method': 'PUT', 'url': f'Patient/{patient.id}'}
            },
            {
                'fullUrl': f'urn:uuid:{practitioner.id}',
                'resource': practitioner,
                'request': {'method': 'PUT', 'url': f'Practitioner/{practitioner.id}'}
            }
        ] + service_requests
    )

    payload = bundle.json()
    print(payload)

    #Sparked Credentials
    username = "placer"
    password = "ps8qs7kLVbjS5Gr"


    response = requests.post(fhir_server_url, data=payload, headers={'Content-Type': 'application/json'}, auth=HTTPBasicAuth(username, password))
    #print(response.text)

    # Pause for 5 seconds
    time.sleep(2)

print('Done')

{"resourceType": "Bundle", "id": "510c7fc8-ea48-4290-b00e-d8ca79ba41f4", "type": "transaction", "entry": [{"fullUrl": "urn:uuid:fd2d7a0c-70be-4db5-81ee-d98e5bfc2d61", "resource": {"resourceType": "Patient", "id": "fd2d7a0c-70be-4db5-81ee-d98e5bfc2d61", "text": {"status": "generated", "div": "<div xmlns='http://www.w3.org/1999/xhtml'>Patient resource</div>"}, "name": [{"text": "Robust Crocodile", "family": "Crocodile", "given": ["Robust"]}], "gender": "female", "birthDate": "1998-08-21", "address": [{"text": "27 Corymbia Pl, Windsor, VIC 3031", "line": ["27 Corymbia Pl"], "city": "Windsor", "state": "VIC", "postalCode": "3031", "country": "Australia"}]}, "request": {"method": "PUT", "url": "Patient/fd2d7a0c-70be-4db5-81ee-d98e5bfc2d61"}}, {"fullUrl": "urn:uuid:3dc32cf6-1508-4d18-9e30-0025c2c7e1e2", "resource": {"resourceType": "Practitioner", "id": "3dc32cf6-1508-4d18-9e30-0025c2c7e1e2", "text": {"status": "generated", "div": "<div xmlns='http://www.w3.org/1999/xhtml'>Practitioner resou